In [1]:
import sys, os
import numpy as np
import pandas as pd
import pandas.io.sql as psql
import sqlalchemy as sql
import sqlite3 as sqlite3

def header(msg):
    print('-'*50 + "\n" + msg +"\n")

filename='GSL_Ops.xlsx'   
conn = sqlite3.connect('customer.db')
cur=conn.cursor()


In [2]:
#header('What’s the revenue of the 90th percentile rank customer_id from CustomerRevenue, ordered by revenue (where the highest-revenue customer_id is in the 99th percentile)?')
sql_problem_1 = ("SELECT '90th Percentile' as Percentile_Rank, min(revenue) as revenue "
                 "FROM (Select CR.customer_id,printf('%.3f',CR.revenue) as revenue, PERCENT_RANK() Over(Order By CR.revenue asc) as rank "
                 "From CustomerRevenue as CR "
                 "Order by CR.revenue asc) "
                 "WHERE rank >=.9 "
                 "Group by 1;").format(1, 2)

print(sql_problem_1)
pd.read_sql_query(sql_problem_1,conn)




SELECT '90th Percentile' as Percentile_Rank, min(revenue) as revenue FROM (Select CR.customer_id,printf('%.3f',CR.revenue) as revenue, PERCENT_RANK() Over(Order By CR.revenue asc) as rank From CustomerRevenue as CR Order by CR.revenue asc) WHERE rank >=.9 Group by 1;


,Percentile_Rank,revenue
0,90th Percentile,6426.570


In [3]:
header('What is the revenue per agency (note: Some customer IDs have an agency assigned, in addition to their CustomerSalesTeam. A customer ID has an agency sales team if it both (1) appears in CustomerAgency and (2) the customer ID’s country in CustomerAgency matches the customer ID’s country in CustomerSalesTeam)?')
sql_problem_3 = ("Select CA.agency_name , sum(printf('%.3f',CR.revenue)) as revenue \
                     FROM CustomerAgency CA \
                     INNER JOIN CustomerSalesTeam CST on CA.customer_id = CST.customer_id and CA.country=CST.country \
                LEFT JOIN CustomerRevenue CR on CA.customer_id = CR.customer_id \
                     GROUP BY 1;")
print(sql_problem_3.replace('  ','').replace
pd.read_sql_query(sql_problem_3,conn)


SyntaxError: invalid syntax (<ipython-input-3-3e0e774fa7fc>, line 8)

In [ ]:
header("Google is always running experiments to try to optimize our products. The CustomerExperiments table indicates which customer IDs are in the ALPHA experiment and which are in the BRAVO experiment. (Some customer IDs are enrolled in both.)"
       "\n When comparing total revenue from customer IDs in the ALPHA experiment to total revenue from customer IDs not in the experiment, which cohort has higher revenue?"
       "\n Is revenue from customers enrolled in the ALPHA experiment significantly different than revenue from customers not in the experiment? What about the BRAVO experiment? Explain the test that you used and include any database queries you would have used to generate the inputs required for the test.")

In [ ]:
print("Cohort with with higher")
sql_problem_4 = ("Select CE.experiment_id , sum(printf('%.3f',CR.revenue)) as revenue "
                     "FROM CustomerRevenue CR "
                     "LEFT JOIN CustomerExperiment CE on CR.customer_id = CE.customer_id "
                     "GROUP BY 1;").format(1, 2)
print(sql_problem_4)
pd.read_sql_query(sql_problem_4,conn)

In [ ]:
print("Breakdown of metrics between ALPHA and Bravo")
sql_problem_4 = ("SELECT QUARTILE, "
                     "MIN(CASE WHEN experiment_id = 'ALPHA' then revenue else null end) as quartile_customer_rev_ALPHA, "
                     "MIN(CASE WHEN experiment_id = 'BRAVO' then revenue else null end) as quartile_customer_rev_BRAVO, "
                     "AVG(CASE WHEN experiment_id = 'ALPHA' then revenue else null end) as average_customer_rev_ALPHA, "
                     "AVG(CASE WHEN experiment_id = 'BRAVO' then revenue else null end) as average_customer_rev_BRAVO, "                     
                     "SUM(CASE WHEN experiment_id = 'ALPHA' then revenue else null end) as total_rev_ALPHA, "
                     "SUM(CASE WHEN experiment_id = 'BRAVO' then revenue else null end) as total_rev_BRAVO, "
                     "COUNT(CASE WHEN experiment_id = 'ALPHA' then customer_id else null end) as participants_ALPHA, "
                     "COUNT(CASE WHEN experiment_id = 'BRAVO' then customer_id else null end) as participants_BRAVO "
                     "FROM(SELECT experiment_id, customer_id, revenue, rev_rank, "
                      "CASE WHEN rev_rank >=0 AND rev_rank<.25 then '1st Quartile'"
                           "WHEN rev_rank >=.25 AND rev_rank<.5 then '2nd Quartile' "
                           "WHEN rev_rank >=.5 AND rev_rank<.75 then '3rd Quartile' "
                           "WHEN rev_rank >=.75 AND rev_rank<=1 then '4th Quartile' "
                           "END as Quartile "
                          "FROM(SELECT experiment_id, customer_id, revenue, percent_rank() Over(partition by experiment_id ORDER BY revenue asc) as rev_rank "
                            "FROM(Select CE.experiment_id , CE.customer_id, "
                                 "sum(printf('%.3f',CR.revenue)) as revenue "
                             "FROM CustomerExperiment CE "
                             "LEFT JOIN CustomerRevenue CR on CE.customer_id = CR.customer_id "
                             "GROUP BY 1,2) "
                         "GROUP BY 1,2,3) "                 
                     "GROUP BY 1,2,3,4,5 ORDER BY 1,4,5,2) "
                 "Group by 1;").format(1, 2)
print(sql_problem_4)
pd.read_sql_query(sql_problem_4,conn)